In [1]:
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import random
import copy

La méthode *compute_some_stats* permet d'effectuer des statistiques de bases : la moyenne et l'écart type d'une matrice.

In [3]:
def compute_some_stats(stats):
    """ Compute the mean and the standard deviation
    for a list of list of state vector """
    
    stats = np.matrix(stats)
    mean_x = np.mean(stats, axis=0)
    std_x = np.std(stats, axis = 0)
    
    return mean_x, std_x

La fonction *gillespie* permet de coder l'algorithme de gillespie mais également de donner quelques statistiques grâce à la fonction *compute_some_stats*. 
Nous avons suivi pas à pas l'algorithme étudié en cours mais nous avons du être astucieux pour certaines lignes, par exemple : 

*   Donner en paramètre 2 matrices : une matrice normale comme vue en cours mais aussi une matrice de réactifs seulement, cela règle le problème de certaines réactions qui ont un élement comme produit et comme réactif.
*   Créer une variable temporaire pour le calcul des λ.
*   Utiliser la fonction np.random.choice pour déterminer si une réaction va se passer ou pas (probabilité = lambd_norm).


In [4]:
def gillespie(x0, k, T, V,Vr):
    
    x=copy.deepcopy(x0)
    t=0
    statsX = [copy.deepcopy(x0)]
    
    while t <= T :
        
        lambd=[]
        for i in range(len(V[0])) :
            for j in range(len(V)): 
                tmp = k[i]
                if Vr[j][i]<0 : #c'est un réactif
                    tmp=tmp*x[j]
            lambd.append(tmp)
        Rtot=sum(lambd)
        deltat=float(np.random.exponential(size=1,scale=(1/Rtot)))
        lambd_norm=[lambd[i]/Rtot for i in range(len(lambd))]
        a=[i for i in range(len(V[0]))]
        for l in range(len(x)) : 
            x[l]=x[l]+V[l][np.random.choice(a,p=lambd_norm)]
        t = t+deltat
        
        statsX.append(copy.deepcopy(x)) # stocke l'etat des réactifs à chaque itération
        
        mean_x, std_x = compute_some_stats(statsX)
        
    return x, mean_x, std_x

In [5]:
from scipy import stats

Pour la fonction tau_leaping, nous avons ré-utiliser celle de Gillespie puis nous l'avons modifiée un petit peu comme vue en cours.

In [6]:
def tau_leaping(x0, k, V, T, tau, v):
    #r=réactions r1 à rj ayant chacune une constante de vitesse k et une reaction rate lambda (kj, lambdaj)
    #V = matrice de changement du à chaque réaction
    #v=vecteur de chanchement d'état pour chaque réactif
    statsX = [copy.deepcopy(x0)]
    x=x0
    t=0
    
    while t <= T :
        #Compute reaction rates lambda
        lambd=[]
        for i in range(len(V[0])) :
            for j in range(len(V)): 
                tmp = k[i]
                if V[j][i]<0 : #c'est un réactif
                    tmp=tmp*x[j]
            lambd.append(tmp)
        Rtot=sum(lambd)
        lambd_norm=[lambd[i]/Rtot for i in range(len(lambd))]

        #for all vi in v : vi <- 0
        for i in range(len(v)):
            v[i]=0
        #Pour chaque réaction, on calcule le Kri, nombre de fois que la réaction k se produit entre t et tau 
        #suit une loi de poisson de paramètre lambdai*tau
        for i in range(len(V[0])):#pour chaque réaction
            Kri=stats.poisson.rvs(mu = lambd[i]*tau, size = 1) 
            #v <- v + Kri*V.j
            for j in range(len(v)):
                v[j]=int(v[j]+Kri*V[j][i])
        for i in range(len(x)): 
            if x[i]+v[i]>0:
                x[i]+=v[i]
            else:
                break
        t = t+tau
        
        statsX.append(copy.deepcopy(x))
        
        mean_x, std_x = compute_some_stats(statsX)
        
    return x, mean_x, std_x

In [7]:
k=[1,3]
x0=[2,1,2]
V=[[-1,1],[-1,1],[1,-1]]
Vr=V

#-1   1
#-1   1
# 1  -1


gillespie(x0,k,2,V,Vr)

([1, -2, 1],
 matrix([[ 2.1, -0.5,  0.7]]),
 matrix([[0.94339811, 1.36014705, 0.64031242]]))

## Exercices 1 & 2 :


#### Ex1 : Implémentation de la méthode gillespie cf. plus haut

#### Ex2 : Simple bio-chemical reactions

In [22]:
x0_ex2=[100,100,100,100,100]
T_ex2=2
#Vr : 
#   R1  R2  R3  R4  R5  R6  R7
# A 0   -1  -1  -1  -1  0   0
# B 0   0   -1   0  -1  0   0
# C 0   0    0   0  0   -1  -1
# D 0   0    0   0  0   0   0
# E 0   0    0   0  0   0   0

#V :  
#   R1  R2  R3  R4  R5  R6  R7
# A 1   -1  -1  0   -1  1   0
# B 0   0   -1  1   -1  1   0
# C 0   0    1  0    1  -1  -1
# D 0   0    0  0    0  0   1
# E 0   0    0  0    0  0   1

k1= 0.1
k2 = 0.05
k = 0.2
k_ex2=[k1, k2,k, k, k1, k1, k2]

V_ex2=[[1,-1,-1,0,-1,1,0],[0,0,-1,1,-1,1,0],[0,0,1,0,1,-1,-1], [0,0,0,0,0,0,1],[0,0,0,0,0,0,1]]
Vr_ex2=[[0,-1,-1,-1,-1,0,0],[0,0,-1,0,-1,0,0],[0,0,0,0,0,-1,-1], [0,0,0,0,0,0,0],[0,0,0,0,0,0,0]]

X2, mean_X2, std_X2 = gillespie(x0_ex2,k_ex2, 2, V_ex2, Vr_ex2)
print ("Final state of the molecules : ", X2, "\n")
print ("Mean of number of molecules : ", mean_X2, "\n")
print ("Standart deviation : ", std_X2)

Final state of the molecules :  [99, 99, 101, 100, 100] 

Mean of number of molecules :  [[ 99.5  99.5 100.5 100.  100. ]] 

Standart deviation :  [[0.5 0.5 0.5 0.  0. ]]


In [9]:
# Equations for the molecules (ODE) :

#R1 
# dAdt = k1

#R2 
# dAdt = -k2[A]

#R3
# dAdt = -k[A]
# dBdt = -k[B]
# dCdt = k[A] + k[B] = k[C]

#R4
# dAdt = -k[A] + k[A] = 0
# dBdt = k[A]

#R5
# dAdt = -k1[A]
# dBdt = -k1[B]
# dCdt = k1[A] + k1[B] = k1[C]

#R6
# dAdt = k1[C]
# dBdt = k1[C]
# dCdt = -k1[C]

#R7
# dCdt = -k2[C]
# dDdt = k2[C]
# dEdt = k2[C]


# Resulting of the 7 reactions :

# dAdt = k1 -k2[A] -k[A] -k1[A] + k1[C]
# dBdt = -k[B]+k[A]-k1[B]+k1[C]
# dCdt = k[C] + k1[C] -k1[C] - k2[C] 
# dDdt = k2[C]
# dEdt = k2[C]


On compare avec les résultats d'un systeme d'équations différentielles.

In [10]:
from scipy.integrate import odeint 

In [11]:
def dX_dt(x,t, k1, k2, k):
    return (k1*1 - k2*x[0] - k*x[0] - k1*x[0] + k1*x[2],
           -k*x[1] + k*x[0] -k1*x[1] + k1*x[2] ,
           k*x[0] + k*x[1] + k1*x[0] + k1*x[1] - k1*x[2] - k2*x[2],
           k2*x[2],
           k2*x[2])

In [30]:
ts=np.linspace(0, 10, 10)
x0 = [100, 100, 100, 100, 100]
k1=0.1
k2 = 0.05
k=0.2
solx = odeint(dX_dt, x0, ts, args=(k1, k2, k))
print(solx)

[[100.         100.         100.         100.         100.        ]
 [ 79.26500267  99.95870168 142.44869023 106.79928146 106.79928146]
 [ 68.53261528 100.50284085 173.78261478 115.62324    115.62324   ]
 [ 63.78511504 102.10621847 198.36211097 125.98431717 125.98431717]
 [ 62.61951488 104.85199702 218.99548377 137.59100538 137.59100538]
 [ 63.61293038 108.66285995 237.48546114 150.27784584 150.27784584]
 [ 65.93301793 113.41398583 254.98884584 163.96025648 163.96025648]
 [ 69.10051287 118.98403844 272.25108232 178.60563048 178.60563048]
 [ 72.84554388 125.27465738 289.75798823 194.21500532 194.21500532]
 [ 77.02137643 132.21488481 307.83316321 210.81154618 210.81154618]]


On fonction des paramètre de constantes d'équilibres $k$ que l'on choisit nous pouvons tomber sur des résultats similaires à ceux observés en utilisant Gillespie.

## Exercice 3 : Gene expression

In [13]:
x0_ex3=[100,100,100,100]

#Vr
#           R1 R2 R3 R4 R5 R6
# dna*      -1 0  0  0  0  0
# dna       0  -1 -1 0  0  0
# rna       0  0  0  -1 -1  0
# protein   0  0  0  0   0  -1

#V
#           R1 R2 R3 R4 R5 R6
# dna*      -1 1  0  0  0  0
# dna       1  -1 0  0  0  0
# rna       0  0  1  0  -1 0
# protein   0  0  0  1  0  -1

k_ex3=[k1,k1,k,k,k,k]

Vr_ex3=[[-1,0,0,0,0,0], [0,-1,-1,0,0,0],[0,0,0,-1,-1,0],[0,0,0,0,0,-1]]
V_ex3= [[-1,1,0,0,0,0], [1,-1,0,0,0,0],[0,0,1,0,-1,0],[0,0,0,1,0,-1]]

gillespie(x0_ex3,k_ex3,2,V_ex3,Vr_ex3)

([101, 100, 100, 65],
 matrix([[100.08333333, 100.        , 100.        ,  82.5       ]]),
 matrix([[ 0.2763854 ,  0.        ,  0.        , 10.38829469]]))

On peut voir que les colonnes 2 et 3 ont les mêmes statistiques.

## Exercice 4 : Lotka-Volterra

In [14]:
x0_ex4=[100,100]

#V
#         R1   R2   R3
#prey     1    -1   0
#predator 0    1   -1

#Vr
#         R1   R2   R3
#prey     0    -1   0
#predator 0    -1   -1

k_ex4=[k1,k2,k]

V_ex4=[[1,-1,0],[0,1,-1]]
Vr_ex4=[[0,-1,0],[0,-1,-1]]

gillespie(x0_ex4,k_ex4,2,V_ex4,V_ex4)

([100, 67], matrix([[100. ,  83.5]]), matrix([[0.        , 9.81070844]]))

## Exercice 5 : Bifurcation

In [15]:
V_ex5=[[1,0,-1,1,0],[0,1,0,0,-1],[0,0,0,0,1],[0,0,0,0,1]]
Vr_ex5=[[0,0,-1,0,0],[0,0,0,0,-1],[0,0,-1,-2,-1],[0,0,0,0,0]]

k_ex5=[k, k, k1, k1, k2]
x0_ex5=[100, 100, 100, 100]

gillespie(x0_ex5,k_ex5,2,V_ex5,Vr_ex5)

([99, 100, 100, 100],
 matrix([[ 99.5, 100. , 100. , 100. ]]),
 matrix([[0.5, 0. , 0. , 0. ]]))

Le système met en évidence une bifurcation de Hopf.

## Exercice 6 : SIR system

In [16]:
# 0 --(k1)--> S 
# S --(k2)--> I
# I --(k3)--> R
# I --(k4)--> 0 

x0_ex6 = [100,100,100]

#V

#     R1   R2   R3   R4
#  S  1    -1   0    0
#  I  0    1    -1   -1
#  R  0    0    1    0

#Vr

#     R1   R2   R3   R4
#  S  0    -1   0    0
#  I  0    0    -1   -1
#  R  0    0    0    0

V_ex6=[[1,-1,0,0], [0,1,-1,-1],[0,0,1,0]]

Vr_ex6 =[[0,-1,0,0], [0,0,-1,-1],[0,0,0,0]]

k1 = 1/3
k2 = 0.75
k3 = 0.5
k4 = 0.1

k_ex6 = [k1,k2,k3,k4]

gillespie(x0_ex6,k_ex6,10,V_ex6,Vr_ex6)

([96, 107, 105],
 matrix([[ 98.54545455, 102.09090909, 102.77272727]]),
 matrix([[1.72487872, 2.92184703, 1.53539507]]))

In [17]:
x0_ex6 = [100,100,100]
V_ex6=[[1,-1,0,0], [0,1,-1,-1],[0,0,1,0]]
k_ex6 = [k1,k1,k2,k]
v=[1,1,1]
tau_leaping(x0_ex6, k_ex6, V_ex6, 2, 0.5, v)

([101, 94, 106],
 matrix([[100.5,  97. , 103. ]]),
 matrix([[0.5      , 2.1602469, 2.1602469]]))